# Generalized-Controllable Variational Autoencoder

### import dependencies

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#------------------basic dependencies --------------------
seed = 42
import os
import random
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import tensorflow_datasets as tfds
from train_gcvae_2d import train_gcvae as gcvae #change this to train_gcvae_2d if you are working with GCVAE
#----import trainer for distributed training
#from train_gcvae_distrib import train_gcvae_distrib
from sklearn.model_selection import train_test_split
from utils import plot_latent_space, compute_metric, model_saver, model_saver_2d
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10})
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rcParams['figure.dpi'] = 120
import seaborn as sns
#set random seed
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
warnings.filterwarnings("ignore")

### import data

In [4]:
#declare path
path = os.getcwd()

#import data
datatype = "mnist"

batch_size = 64
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() #load data
N, L, M = x_train.shape #extract features of the data
x_train = x_train.reshape(-1, L, M, 1).astype('float32') / x_train.max() #normalize training data
x_train, _ = train_test_split(x_train, test_size = .999, random_state = 42) #split into train/test keeping only small fraction
x_test = x_test.reshape(-1, L, M, 1).astype('float32') / x_train.max() #normalize test data
x_test, _ = train_test_split(x_test, test_size = .992, random_state = 42) #split into train/test keeping only small fraction

train_dataset = tf.data.Dataset.from_tensor_slices(x_train) #convert data into tensor
train_dataset = train_dataset.shuffle(buffer_size = 1024).batch(batch_size)

#test data
test_dataset = tf.data.Dataset.from_tensor_slices(x_test)
test_dataset = test_dataset.shuffle(buffer_size = 1024).batch(batch_size)